In [11]:
import h5py
import pandas as pd
import json
import numpy as np
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import scipy.sparse as ssp
from sklearn.preprocessing import normalize
from torch.utils.data import Dataset, DataLoader
from sklearn.utils import shuffle
import pickle
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import torch
import numpy as np
import itertools
from torch.utils.data import Dataset, DataLoader
from scipy.sparse import coo_matrix, csr_matrix
from collections import Counter
import pickle
from sklearn.utils import shuffle
from os.path import join as pjoin
from tqdm import tqdm
import os
import joblib
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
import importlib 
from sklearn.preprocessing import normalize as sk_normalize
import pytorch_lightning as pl
from os.path import join as pjoin

from scipy import sparse

In [12]:
torch.cuda.set_device(0)

In [13]:
import tqdm as tt

In [14]:
iddf = lambda x: x

In [15]:
import mlps

In [16]:
wd = "submit_dataset/"
loads = ['tr_ds', 'val_ds', 'val_csv','tr_csv', 
         'leaderboard_ds', 'final_ds', 'leaderboard_csv', 
         'final_csv', 'id2idx', 'cv']
rets = []
for l in loads:
    rets.append(joblib.load(wd + l))
tr_02_ds, val_ds, val_csv, tr_csv_02, l_ds, f_ds, l_csv, f_csv, id2idx, cv = rets

In [17]:
idx2id = {y:x for (x,y) in id2idx.items()}
n_items = len(id2idx)

In [39]:
final_idmap = joblib.load('../processed_submit/indices')[0]
idmap = [final_idmap[idx2id[x]] for x in range(len(idx2id))]

In [9]:
import mlps
importlib.reload(mlps)


lr = 1.5 * 1e-4

def get_model(dim=256):
    return mlps.MLP(tr_02_ds.matrices, 
                    tr_02_ds.categoricals,  
                    tr_02_ds.tr_scalar,
                    n_items=n_items, 
                    dim=np.random.choice([256]), 
                    layer_dim=np.random.choice([3000]),
                    dropout=np.random.choice([0.3, 0.35]),
                    use_cat=False,
                    num_feats=904)

softmax_models = [get_model() for x in range(10)]
def get_opts(model):
    wd = np.random.choice([1e-3, 3e-3])
    return torch.optim.Adam(model.parameters(),  lr=lr, weight_decay=wd)

opts = [get_opts(model) for model in softmax_models]
for m in softmax_models:
    m.cuda()
    m.train()
lf = torch.nn.CrossEntropyLoss()

item_last_20
item_last_5
item_last_2
item_last_1
item_last_5_feats
item_last_3_feats
item_last_1_feats
item_last_20
item_last_5
item_last_2
item_last_1
item_last_5_feats
item_last_3_feats
item_last_1_feats
item_last_20
item_last_5
item_last_2
item_last_1
item_last_5_feats
item_last_3_feats
item_last_1_feats
item_last_20
item_last_5
item_last_2
item_last_1
item_last_5_feats
item_last_3_feats
item_last_1_feats
item_last_20
item_last_5
item_last_2
item_last_1
item_last_5_feats
item_last_3_feats
item_last_1_feats
item_last_20
item_last_5
item_last_2
item_last_1
item_last_5_feats
item_last_3_feats
item_last_1_feats
item_last_20
item_last_5
item_last_2
item_last_1
item_last_5_feats
item_last_3_feats
item_last_1_feats
item_last_20
item_last_5
item_last_2
item_last_1
item_last_5_feats
item_last_3_feats
item_last_1_feats
item_last_20
item_last_5
item_last_2
item_last_1
item_last_5_feats
item_last_3_feats
item_last_1_feats
item_last_20
item_last_5
item_last_2
item_last_1
item_last_5_feats
item_l

In [10]:
def validate_models(models, val_ds, use_filter=True, skip_val_only=False):
    hr = []
    for model in models:
        model = model.eval()
    s = 0
    tot_mrr = []
    ii = 0
    for input in DataLoader(val_ds, batch_size=250, shuffle=True):
        (target_idx, _), (ret_mat, ret_cat, ret_scalar) = input
        for k in ret_mat:
            ret_mat[k] = ret_mat[k].cuda()
        for k in ret_cat:
            ret_cat[k] = ret_cat[k].cuda()
        ret_scalar = ret_scalar.cuda()
        rret = []
        for model in models:
            ret = model.forward(ret_mat, ret_cat, ret_scalar,)
            rret.append(ret)
        ret = torch.stack(rret)
        ret = torch.mean(ret, 0)
        ret[ret_mat['item_bow'].bool()] = -10000.0
        top_rec = (-ret).argsort(-1)[:, :100].detach().cpu()

#         if skip_val_only:
#         top_rec = top_rec[target_idx != 29999]
#         target_idx = target_idx[target_idx != 29999]

        mrr = (top_rec == target_idx.unsqueeze(1)).float().numpy()
        hitst = mrr[:, :5].sum(-1)
        mrr = (mrr / np.expand_dims(np.arange(1, 1 + 100), 0)).sum(-1)
        tot_mrr.extend(mrr.tolist())
        hr.extend(hitst)
    for model in models:
        model = model.train()
    return np.mean(tot_mrr), np.mean(hr)

In [11]:
mrr, hr = validate_models(softmax_models, val_ds)

In [12]:
tr_dl = DataLoader(tr_02_ds,
                   batch_size=256, shuffle=False,
                   num_workers=8, prefetch_factor=2,
#                    persistent_workers=True,
                   pin_memory=True)

loss_ = np.log(n_items)
max_mrr = -1
for epoch in range(30):
    it = 0
    iidx = 0
    pbar = tqdm(tr_dl)
    for x in pbar:
        (target_idx, mask), (mats, cats, scalars) = x
        target_idx = target_idx.cuda()
        mask = mask.cuda().squeeze(1)
        
        for k in mats:
            mats[k] = mats[k].cuda()
        for k in cats:
            cats[k] = cats[k].cuda()
#         cats = {}
        scalars = scalars.cuda()
        for model, optimizer in zip(softmax_models, opts):
            model.zero_grad()
            pred = model(mats, cats, scalars)
            pred[mask.bool()] = -10000.0
            ce_loss = lf(pred, target_idx).mean()        
            loss = ce_loss
            loss.mean().backward()
            optimizer.step()
        loss_ = 0.99 * loss_  + 0.01 * loss.detach().cpu().numpy()
        pbar.set_postfix({'loss:': "%0.4f" % loss_})

    model = softmax_models[0]
    model = model.eval()
    mrr, hr = validate_models(softmax_models, val_ds)
    
    if epoch < 20:
        continue
    
    if mrr >= max_mrr:
        torch.save(softmax_models, 'submit_softmax_model_no_shuffle')
        max_mrr = mrr 
    print("[ITER %d] VAL MRR:" % (1 + epoch), mrr, "HR", hr)
    model = model.train()
    model.cuda()


100%|████████████████████████████████████████████████████████████████████████████████████████████| 1738/1738 [05:38<00:00,  5.13it/s, loss:=5.0833]


[ITER 21] VAL MRR: 0.32402888583981676 HR 0.43


100%|████████████████████████████████████████████████████████████████████████████████████████████| 1738/1738 [05:38<00:00,  5.14it/s, loss:=5.0837]


[ITER 22] VAL MRR: 0.3242977234834082 HR 0.43166667


100%|████████████████████████████████████████████████████████████████████████████████████████████| 1738/1738 [05:40<00:00,  5.11it/s, loss:=5.0843]


...

[ITER 28] VAL MRR: 0.3242955147345098 HR 0.43


100%|████████████████████████████████████████████████████████████████████████████████████████████| 1738/1738 [05:34<00:00,  5.19it/s, loss:=5.0791]


[ITER 29] VAL MRR: 0.3249265835189773 HR 0.42866668


100%|████████████████████████████████████████████████████████████████████████████████████████████| 1738/1738 [05:35<00:00,  5.17it/s, loss:=5.0813]


[ITER 30] VAL MRR: 0.3240898431365562 HR 0.42966667


softmax_models = torch.load('submit_softmax_model')
softmax_models = [m.cuda().eval() for m in softmax_models]

In [40]:
softmax_models = torch.load('submit_softmax_model_no_shuffle')

In [41]:
softmax_models = [model.cuda().eval() for model in softmax_models]

In [42]:
mrr, hr = validate_models(softmax_models, val_ds)

In [43]:
mrr, hr

(0.32492658351897724, 0.42866668)

In [44]:
l_dl = DataLoader(l_ds, batch_size=250, shuffle=False, num_workers=8)

In [45]:
ret = []
softmax_models = [model.cuda().eval() for model in softmax_models]
for x in tqdm(l_dl):
    (target_idx, mask), (mats, cats, scalars) = x
    target_idx = target_idx.cuda()
    mask = mask.cuda().squeeze(1)

    for k in mats:
        mats[k] = mats[k].cuda()
    for k in cats:
        cats[k] = cats[k].cuda()
#         cats = {}
    scalars = scalars.cuda()
    preds = []
    for model in softmax_models:
        model.zero_grad()
        pred = model(mats, cats, scalars)
        preds.append(pred)
    pred = torch.stack(preds, 0).mean(0)
    pred[mask.bool()] = -100000.0
    ret.append(pred.cpu().detach().numpy())

100%|██████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:26<00:00,  7.64it/s]


In [46]:
ret = np.vstack(ret)

In [47]:
def new_logit_map(logits, idmap):
    new_map = np.ones(shape=(logits.shape[0], np.max(list(final_idmap.values())) + 1)) * -100000.0
    idmap = [final_idmap[idx2id[x]] for x in range(len(idx2id))]
    new_map[:, idmap] = logits
    return new_map

In [48]:
leaderboard_logits = new_logit_map(ret, idmap)

In [49]:
f_dl = DataLoader(l_ds, batch_size=250, shuffle=False, num_workers=8)

In [50]:
ret = []
softmax_models = [model.cuda().eval() for model in softmax_models]
for x in tqdm(f_dl):
    (target_idx, mask), (mats, cats, scalars) = x
    target_idx = target_idx.cuda()
    mask = mask.cuda().squeeze(1)

    for k in mats:
        mats[k] = mats[k].cuda()
    for k in cats:
        cats[k] = cats[k].cuda()
#         cats = {}
    scalars = scalars.cuda()
    preds = []
    for model in softmax_models:
        model.zero_grad()
        pred = model(mats, cats, scalars)
        preds.append(pred)
    pred = torch.stack(preds, 0).mean(0)
    pred[mask.bool()] = -100000.0
    ret.append(pred.cpu().detach().numpy())

100%|██████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:28<00:00,  6.94it/s]


In [51]:
ret = np.vstack(ret)

In [52]:
final_logits = new_logit_map(ret, idmap)

In [53]:
import time
time.time()

1657099744.6056175

In [62]:
!rm -rf ../logits/leader/mlp-*

In [63]:
!rm -rf ../logits/final/mlp-*

In [66]:
final_logits.shape

(50000, 23691)

In [65]:
import os
os.makedirs("../logits/leader", exist_ok=True)
os.makedirs("../logits/final", exist_ok=True)
joblib.dump(
    (l_csv.session_id.tolist(), leaderboard_logits),
    "../logits/leader/mlp-noshuffle-%d" % int(time.time()))

joblib.dump(
    (f_csv.session_id.tolist(), final_logits),
    "../logits/final/mlp-noshuffle-%d" % int(time.time()))

['../logits/final/mlp-noshuffle-1657099888']

cands = pd.read_csv('../data/candidate_items.csv', dtype=str).item_id.tolist()
cands = set(cands)

len(cands)

_ret = ret + ret2

ls "/data/project/rw/recsys2022/ensemble/mlp/"

topks = (-_ret).argsort(-1)[:, :300]

df = {
    'session_id': [],
    'item_id': [],
    'rank': [],
}
for sid, topk in  zip(l_csv.session_id, topks):
#     print(sid, recs)
    recs = [idx2id[x] for x in topk]
    recs = [x for x in recs if x in cands][:100]
    if len(recs) < 100:
        print(len(recs))
    for rank, rec in zip(range(1, 101), recs):
        df['session_id'].append(sid)
        df['item_id'].append(rec)
        df['rank'].append(rank)

df = pd.DataFrame(df)

df['session_id'] = df['session_id'].apply(lambda x: int(x))

df = df.sort_values(by=['session_id', 'rank'])

df.to_csv("df_noshuffle_shuffle_together.csv", index=False)

sample = pd.read_csv("../sample.csv", dtype=str).sort_values(by=['session_id', 'rank']).reset_index(drop=True)

df = pd.read_csv("df_1.csv", dtype=str).sort_values(by=['session_id', 'rank']).reset_index(drop=True)

a = set(sample['item_id'].tolist()) - cands
len(a)

df.to_csv("ret",index=False)

ls -al /data/project/rw/recsys2022/recsys2022.git/data